<img src="../Img/Python_SQL_PI.png" style="width:100%">

<p style="font-size:180%;text-align:center">Лекция 6. Библиотека SQLAlchemy 2.0</p>

<br><br>
21 апреля 2022 года, чт., В4/3501 (4-й Вешняковский проезд, 4), 6-я пара, 17:20 - 18:50 <br>
ПИ20-1, ПИ20-2, ПИ20-3 <br>

25 апреля 2022 года, пн., Щ38/512 (ул. Щербаковская, 38), 1-я пара, 08:30 - 10:00 <br>
ПИ20-4, ПИ20-5, ПИ20-6 <br>

Смирнов Михаил Викторович, доцент Департамента анализа данных и машинного обучения Финансового университета при Правительстве Российской Федерации. mvsmirnov@fa.ru

<p style="text-align:center">Москва - 2022</p>

При подготовке материалов учебных занятий использовались источники
- Essential SQLAlchemy: Mapping Python to Databases 2nd Edition. Jason Myers, Rick Copeland. O'Reilly Media, Inc. 2015.
- Официальный сайт СУБД sqlite https://sqlite.org
- Официальный сайт библиотеки sqlalchemy https://www.sqlalchemy.org/


SQLAlchemy реализует два API: Core — ядро и ORM - объектно-реляционное сопоставление.

**SQLAlchemy Core** является базовой архитектурой для SQLAlchemy как «набора инструментов для работы с базами данных». Библиотека предоставляет инструменты для управления подключением к базе данных, взаимодействия с запросами и результатами базы данных, а также для программного использования SQL-выражений.

**SQLAlchemy ORM** основывается на Core, предоставляя дополнительные возможности реляционного сопоставления объектов.
ORM предоставляет дополнительный уровень конфигурации, позволяющий сопоставлять пользовательские классы Python с таблицами базы данных и другими конструкциями, а также механизм, известный как сеанс. Затем он расширяет язык выражений SQL базового уровня, позволяя составлять и вызывать SQL-запросы в терминах пользовательских объектов.

<Table align=left>
<tr><td bgcolor=#0dcaf0></td><td style="text-align:left;font-size:10pt">
Таким цветом слева будут отмечены сведения, принципиально важные для ORM-подхода.
</td></tr></Table>

<br><br>
<a id = Home></a>

<p style="font-size:150%"> Оглавление </p>

1. [Механизм подключения к базе данных](#1)
2. [Транзакции и программный интерфейс приложения](#2)
3. [Метаданные базы данных](#3)
4. [Работа с данными](#4)
4. [Объектно-реляционное сопоставление](#5)
5. [Связанные объекты](#6)
6. [Литература](#7)

<p style="font-size:150%"> Проверка версии SQLAlchemy </p>

Для выполнения кода примеров требуется версия SQLAlchemy 1.4.

In [1]:
# Проверка версии SQL Alchemy
import sqlalchemy
sqlalchemy.__version__

'1.4.22'

<p style="font-size:150%"> Версия 1.4 в стиле 2.0 </p>

Планируемая следующая версия SQLAlchemy — это верстия 2.0. В версии 1.4 уже реализован интерфейс в стиле 2.0, и чтобы его использовать, введен параметр `future`.

<a id=1></a>

[К оглавлению](#Home)

# 1. Механизм подключения к базе данных

Начало любого приложения SQLAlchemy — это объект, называемый *Engine* (механизм, движок). Этот объект действует как центральный источник подключения к базе данных, предоставляя как движок, так и пространство хранения, называемое пулом подключений к базой данных. Механизм обычно представляет собой глобальный объект, созданный только один раз для определенного сервера базы данных и настроенный с использованием строки URL-адреса хоста базы данных или серверной части.

В этой лекции мы будем использовать базу данных SQLite в памяти. Это простой способ протестировать вещи, не требуя реальной, уже существующей базы данных. Движок создается с помощью функции `create_engine()`. В качестве одного из аргументов используем `future=True`, чтбы начать работу в стиле 2.0.

In [2]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)

Параметр в виде строки URL укзаывает на три вещи:
1. Название СУБД, в данном случае это *sqlite*.
2. Программный интерфейс приложения для работы с БД, *pysqlite*.
3. Название и местонахождение БД, в данном случае в памяти.

<p style="font-size:150%">Ленивое соединение</p>

Создание движка *engine* само по себе не приводит к соединению с базой данных до тех пор, пока движок не получит какое-либо задание.

Мы установили значение парметра *future=True*, чтобы использовать стиль версии 2.0:
Мы также установили параметр *echo=True*, чтобы движок журналировал все SQL команды. 

<a id=2></a>

[К оглавлению](#Home)

# 2. Транзакции и программный интерфейс приложения

Теперь, когда объект *Engine* готов к работе, мы можем изучить базовую работу движка и его основные интерактивные конечные точки, *Connection* и *Result*. Мы дополнительно представим фасад *ORM* для этих объектов, известный как *Session*.

<Table>
<tr><td bgcolor=#0dcaf0></td><td style="text-align:left;font-size:10pt">
При использовании <b>ORM</b> движок управляется другим объектом, называемым <i>сеансом</i> - <b>session</b>. Сеанс в SQLAlchemy подчеркивает транзакционный способ выполнения, который в значительной степени идентичен объекту <i>Connection</i>, обсуждаемому ниже. Поэтому, хотя этот подраздел ориентирован на пользователей <i>Core</i>, все концепции здесь по существу также относятся к использованию <i>ORM</i>. В конце этого раздела рассмотрим отличия шаблона выполнения <i>Connection</i> от шаблона <i>Session</i> .
</td></tr></Table>

Поскольку нам еще только предстоит знакомство с языком выражений *SQLAlchemy*, который является основным для этой библиотеки, мы будем здесь использовать функцию, называемую `text()`, которая позволяет нам писать операторы *SQL* как текст. Отметим, что текстовый SQL в повседневном использовании SQLAlchemy является скорее исключением, чем правилом для большинства задач, хотя он всегда остается полностью доступным.

<p style="font-size:150%"> Организация подключения </p>

Единственная цель объекта Engine с точки зрения пользователя — предоставить подключение к базе данных, называемое *Connection*. При работе с *Core* напрямую объект *Connection* — это то, как выполняется все взаимодействие с базой данных. Поскольку подключение представляет собой открытый ресурс для базы данных, мы хотим всегда ограничивать область использования этого объекта определенным контекстом, и лучший способ сделать это — использовать форму диспетчера контекста Python, также известную как оператор *with*. Проиллюстрируем выполнение выражения текстовой строки «Hello World» с помощью текстового оператора SQL.

In [3]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2022-04-21 10:32:53,408 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:32:53,411 INFO sqlalchemy.engine.Engine select 'hello world'
2022-04-21 10:32:53,412 INFO sqlalchemy.engine.Engine [generated in 0.00435s] ()
[('hello world',)]
2022-04-21 10:32:53,414 INFO sqlalchemy.engine.Engine ROLLBACK


В приведенном примере диспетчер контекста предоставил соединение с базой данных, а также поместил операцию в транзакцию. Поведение Python DBAPI по умолчанию включает в себя то, что транзакция всегда выполняется. Когда операция завершается, выдается ROLLBACK для завершения транзакции. Транзакция не фиксируется автоматически. Когда мы хотим зафиксировать изменения, нам обычно нужно вызвать `Connection.commit()`.

<p style="font-size:150%">Завершение изменений</p>

Итак, объект соединения не завершает изменения по умолчанию. Что надо сделать, чтобы завершить изменения? покажем это на примере. Создадим таблицу, вставим в неё данные, а затем завершим изменения с помощью метода `Connection.commit()` внутри той же области, в которой мы с помощью диспетчера контекста создали объект соединения.

In [4]:
# "commit as you go"
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
                 [{"x": 1, "y": 1}, {"x": 2, "y": 4}])
    conn.commit()

2022-04-21 10:32:53,559 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:32:53,561 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2022-04-21 10:32:53,565 INFO sqlalchemy.engine.Engine [generated in 0.00581s] ()
2022-04-21 10:32:53,570 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-04-21 10:32:53,572 INFO sqlalchemy.engine.Engine [generated in 0.00293s] ((1, 1), (2, 4))
2022-04-21 10:32:53,575 INFO sqlalchemy.engine.Engine COMMIT


Мы создали два оператора SQL, которые обычно являются транзакционными: оператор *CREATE TABLE* и параметризованный оператор *INSERT* (приведенный выше синтаксис параметризации обсуждается несколькими разделами ниже в разделе «Отправка нескольких параметров»). Поскольку мы хотим, чтобы проделанная нами работа была зафиксирована в нашем блоке, мы вызываем метод `Connection.commit()`, который завершает транзакцию. После того, как мы вызовем этот метод внутри блока, мы можем продолжить выполнение других операторов SQL, и, если мы захотим, мы можем снова вызвать `Connection.commit()` для последующих операторов. SQLAlchemy относится к этому стилю как к «фиксации по ходу» выполнения - «commit as you go».

Существует также другой стиль фиксации данных, который заключается в том, что мы можем заранее объявить наш блок «подключения» блоком транзакции. Для этого режима работы мы используем метод `Engine.begin()` для получения соединения, а не метод `Engine.connect()`. Этот метод управляет областью соединения и заключает все действия внутри транзакции с *COMMIT* в случае успеха или *ROLLBACK* в случае возникновения исключения. В дальнейшем этот стиль может именоваться "начать сразу" — *begin once*.

In [5]:
# "begin once"
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}]
    )

2022-04-21 10:32:53,781 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:32:53,782 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-04-21 10:32:53,784 INFO sqlalchemy.engine.Engine [cached since 0.2152s ago] ((6, 8), (9, 10))
2022-04-21 10:32:53,788 INFO sqlalchemy.engine.Engine COMMIT


<Table>
    <tr><td bgcolor=#adb5bd></td><td style="text-align:left; font-size:10pt">

Что значит BEGIN (implicit)?

В начале блока транзакции журнал показал строку «BEGIN (неявно)». «Неявно» здесь означает, что SQLAlchemy считает, что это начало неявной транзакции DBAPI. Например, вы можете зарегистрировать обработчики событий для перехвата этого события.

Дополнительно о событиях: https://docs.sqlalchemy.org/en/14/core/events.html#core-sql-events
</td></tr></Table>

<p style="font-size:150%"> Основы выполнения SQL-выражений </p>

Мы изучили примеры, в которых операторы SQL выполняются с использованием метода `Connection.execute()` в сочетании с объектом  `text()` и возвратом объекта *Result*. Проиллюстрируем более подробно взаимодействие этих компонентов.

<Table>
<tr><td bgcolor=#0dcaf0></td><td style="text-align:left; font-size:10pt">
Большая часть содержимого этого раздела одинаково хорошо применима к ORM при использовании метода Session.execute(), который работает очень похоже на метод Connection.execute(), включая то, что строки результатов ORM доставляются с использованием объекта <a href="https://docs.sqlalchemy.org/en/14/core/connections.html#sqlalchemy.engine.Result"> Result </a>.
        </td></tr>
</Table>

<p style="font-size:120%">Получение строк</p>

Сначала проиллюстрируем объект *Result*, используя строки, которые мы вставили ранее, выполнив оператор *SELECT* для созданной нами таблицы:

In [6]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}  y: {row.y}")

2022-04-21 10:32:53,963 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:32:53,964 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2022-04-21 10:32:53,965 INFO sqlalchemy.engine.Engine [generated in 0.00213s] ()
x: 1  y: 1
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2022-04-21 10:32:53,970 INFO sqlalchemy.engine.Engine ROLLBACK


Команда «SELECT» выбрала все строки из таблицы. Возвращаемый объект *Result* является итерируемым объектом строк результатов.

*Result* имеет множество методов для выборки и преобразования строк, таких как показанный ранее метод `Result.all()`, который возвращает список всех объектов *Row*. Он также реализует интерфейс итератора `Python`, так что мы можем напрямую перебирать коллекцию объектов *Row*.

Сами объекты *Row* должны действовать как <a href="https://docs.python.org/3/library/collections.html#collections.namedtuple"> именованные кортежи </a> Python. Ниже проиллюстрируем различные способы доступа к строкам.

- Позиционное. Этот стиль заключается в назначении переменных каждому атрибуту строки позиционно:

```
result = conn.execute(text("select x, y from some_table"))
for x, y in result:
    print(x, y)
```

- По индексу. Кортежи — это упорядоченные неизменяемые коллекции, к элементам которых можно обращаться обычным образом по целочисленному индексу

```
with engine.connect() as conn:
    result = conn.execute(text("select x, y from some_table"))
    for row in result:
        x, y = row[0], row[1]
        print(x, y)
```
- По имени атрибута. 
Поскольку это именованные кортежи Python, они имеют динамические имена атрибутов, соответствующие именам каждого столбца. Эти имена обычно являются именами, которые оператор SQL присваивает столбцам в каждой строке.

```
with engine.connect() as conn:
    result = conn.execute(text("select x, y from some_table"))
    for row in result:
        print(f"Row: {row.x} {row.y}")
```

- По сопоставлению. Чтобы получать строки в качестве объектов сопоставления Python, что по сути является версией "для чтения" обычного объекта *dict*, *Result* может быть преобразован в объект *MappingResult* с использованием модификатора `Result.mappings()`. Это объект результата, который дает подобные словарю объекты *RowMapping*, а не объекты *Row*:

```
with engine.connect() as conn:
    result = conn.execute(text("select x, y from some_table"))
    for dict_row in result.mappings():
        x, y = dict_row['x'], dict_row['y']
        print(x,y)
```

<p style="font-size:150%">Отправка параметров</p>

Операторы SQL обычно сопровождаются данными, которые должны быть переданы вместе с самим оператором, как мы видели ранее в примере вставки *INSERT*. Таким образом, метод `Connection.execute()` также принимает параметры, которые называются связанными параметрами. Элементарным примером может быть то, что мы хотим ограничить оператор «SELECT» только строками, которые соответствуют определенным критериям, например, строками, в которых значение «y» превышает определенное значение, переданное функции.

Чтобы добиться этого, мы добавляем в выражение условие *WHERE* новый параметр с именем «y»; функция `text()` принимает его, используя формат двоеточия «:y». Фактическое значение «:y» затем передается в качестве второго аргумента функции `Connection.execute()` в виде словаря:

In [7]:
with engine.connect() as conn:
    result = conn.execute(
        text("SELECT x, y FROM some_table WHERE y > :y"),
        {"y": 2}
    )
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-04-21 10:32:54,072 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:32:54,073 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2022-04-21 10:32:54,074 INFO sqlalchemy.engine.Engine [generated in 0.00148s] (2,)
x: 2  y: 4
x: 6  y: 8
x: 9  y: 10
2022-04-21 10:32:54,075 INFO sqlalchemy.engine.Engine ROLLBACK


В журналированном выводе SQL мы видим, что связанный параметр `:y` преобразован в вопросительный знак, когда он был отправлен в базу данных SQLite. Это связано с тем, что драйвер базы данных SQLite использует формат передачи параметров в стиле вопросительного знака, который является форматом, разрешенным спецификацией DBAPI.

<Table><tr><td bgcolor=#adb5bd></td><td style="text-align:left; font-size:10pt">
Как упоминалось в начале этого раздела, текстовый SQL не является обычным способом работы с SQLAlchemy. Однако при использовании текстового SQL литеральные значения Python, даже нетекстовые, такие как целые числа или даты, никогда не должны напрямую преобразовываться в строку SQL; всегда следует использовать параметр. Это наиболее известно как способ избежать атак с помощью SQL-инъекций. Однако это также позволяет диалектам SQLAlchemy и/или DBAPI правильно обрабатывать входящие входные данные для серверной части. За исключением случаев использования простого текстового SQL, SQLAlchemy Core Expression API гарантирует, что литеральные значения Python передаются в качестве связанных параметров, где это необходимо.
</td></tr></Table>

<p style="font-size:150%"> Отправка нескольких параметров </p>

В примере с фиксацией изменений мы выполнили оператор INSERT, где оказалось, что мы можем вставить сразу несколько строк в базу данных. Для операторов, которые работают с данными, но не возвращают наборы результатов, а именно операторов DML, таких как «INSERT», мы можем отправить в метод `Connection.execute()` несколько параметров в списке словарей:

In [8]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 11, "y": 12}, {"x": 13, "y": 14}]
    )
    conn.commit()

2022-04-21 10:32:54,191 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:32:54,194 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2022-04-21 10:32:54,197 INFO sqlalchemy.engine.Engine [cached since 0.6272s ago] ((11, 12), (13, 14))
2022-04-21 10:32:54,200 INFO sqlalchemy.engine.Engine COMMIT


На самом деле, объекты *Connection* используют возможность DBAPI, известную как `cursor.executemany()`. Этот метод выполняет эквивалентную операцию вызова данного оператора SQL для каждого набора параметров в отдельности. DBAPI может оптимизировать эту операцию различными способами, используя подготовленные операторы или в некоторых случаях объединяя наборы параметров в один оператор SQL. Некоторые диалекты SQLAlchemy могут также использовать альтернативные API для этого случая, такие как диалект *psycopg2* для *PostgreSQL*, который в таких случаях использует более производительные API.

<Table align=left>
<tr><td bgcolor=#0dcaf0></td><td style="text-align:left;font-size:10pt">
Интерфейс ORM использует другой подход передачи нескольких параметров операторам INSERT/UPDATE/DELETE.
</td></tr></Table>

<br><p>
<p style="font-size:150%">Связывание параметров с операторами SQL-выражения</p>

Два предыдущих примера иллюстрируют ряд параметров, передаваемых вместе с оператором SQL. Для выполнения выражений с одним параметром, использование параметров в SQLAlchemy на самом деле чаще всего осуществляется путем связывания с оператором, что является основной функцией языка выражений SQL и позволяет создавать запросы, которые могут быть составлены естественным образом. Эта концепция будет обсуждаться более подробно в следующих разделах. Предварительно отметим, что сама конструкция *text()*, являющаяся частью языка выражений SQL, поддерживает эту функцию с помощью метода `TextClause.bindparams()`; это <a href="https://docs.sqlalchemy.org/en/14/glossary.html#term-generative">генеративный</a> метод, который возвращает новую копию конструкции SQL, дополненную значениями параметров, которые мы хотим передать:

In [9]:
stmt = text("SELECT x, y FROM some_table WHERE (y > :y) AND (x > :x) ORDER BY x, y").bindparams(y=6, x=0)
with engine.connect() as conn:
    result = conn.execute(stmt)
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-04-21 10:32:54,299 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:32:54,300 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE (y > ?) AND (x > ?) ORDER BY x, y
2022-04-21 10:32:54,300 INFO sqlalchemy.engine.Engine [generated in 0.00150s] (6, 0)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2022-04-21 10:32:54,303 INFO sqlalchemy.engine.Engine ROLLBACK


Отметим, что в результате связывания параметра с SQL-выражением, метод `Connection.execute()` сформировал как строку SQL, так и отдельный кортеж параметров, в данном случае, из двух элементов.

<p style="font-size:150%">Выполнение сеанса ORM</p>

<Table align=left>
<tr><td bgcolor=#0dcaf0></td><td style="text-align:left;font-size:10pt">
Как упоминалось ранее, большинство приведенных выше шаблонов и примеров также применимы для объектно-реляционного сопоставления ORM.
</td></tr></Table><br><br>

Основным объектом при использовании ORM является сеанс — *Session*. В SQLAlchemy этот объект используется способом, похожим на метод *Connection*, и фактически, когда используется сеанс, он ссылается на внутреннее подключение *Connection*, которое он использует для генерации SQL-выражения.

Сеанс имеет несколько различных шаблонов создания, проиллюстрируем здесь базовый, который отслеживает то, как используется соединение. Создадим сеанс в диспетчере контекста:

In [10]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y").bindparams(y=6)
with Session(engine) as session:
    result = session.execute(stmt)
    for row in result:
       print(f"x: {row.x}  y: {row.y}")

2022-04-21 10:32:54,493 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:32:54,493 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2022-04-21 10:32:54,493 INFO sqlalchemy.engine.Engine [generated in 0.00037s] (6,)
x: 6  y: 8
x: 9  y: 10
x: 11  y: 12
x: 13  y: 14
2022-04-21 10:32:54,507 INFO sqlalchemy.engine.Engine ROLLBACK


Сравним этот пример с примером в предыдущем параграфе «Связывание параметров с оператором». Мы заменяем вызов `engine.connect() as conn` на `Session(engine) as session`, а затем используем `Session.execute()` точно так же, как и с методом `Connection.execute()`.

Кроме того, как и *Подключение*, *Сеанс* следует шаблону поведения «фиксации по ходу» с использованием метода `Session.commit()`, показанного ниже с использованием  SQL-оператора *UPDATE*.

In [11]:
with Session(engine) as session:
    result = session.execute(
        text("UPDATE some_table SET y=:y WHERE x=:x"),
        [{"x": 9, "y":11}, {"x": 13, "y": 15}]
    )
    session.commit()

2022-04-21 10:32:54,527 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:32:54,528 INFO sqlalchemy.engine.Engine UPDATE some_table SET y=? WHERE x=?
2022-04-21 10:32:54,529 INFO sqlalchemy.engine.Engine [generated in 0.00125s] ((11, 9), (15, 13))
2022-04-21 10:32:54,530 INFO sqlalchemy.engine.Engine COMMIT


В этом примере мы вызвали оператор UPDATE, используя связанный параметр в стиле «executemany» и заканчивая блок фиксацией по ходу «commit as you go».

<Table align="left">
    <tr><td bgcolor=#0dcaf0></td>
        <td style="text-align:left;font-size:10pt">
Примечание. Сеанс не удерживает подключение Connection после завершения транзакции. Он получает новое соединение от движка, когда в следующий раз требуется выполнение SQL-запроса к базе данных.
        </td></tr>
</Table>

<br><br><p>
У Сеанса, очевидно, значительно больше особенностей, однако понимание того, что метод `Session.execute()` используется так же, как `Connection.execute()`, поможет нам изучить примеры, которые последуют позже.
</p>

Дополнительно: <a href="https://docs.sqlalchemy.org/en/14/orm/session_basics.html#id1">Основы Сеанса</a>

<a id=3></a>

[К оглавлению](#Home)

# 3. Метаданные базы данных

Центральным элементом обоих интерфейсов SQLAlchemy — как Core, так и ORM, является язык выражений SQL, предназначенный для SQL-запросов. Основой для этих запросов являются объекты Python, представляющие такие понятия базы данных, как таблицы и столбцы. Эти объекты вместе известны как <a href="https://docs.sqlalchemy.org/en/14/glossary.html#term-database-metadata">метаданные базы данных</a>.

Наиболее распространенные базовые объекты <a href="https://docs.sqlalchemy.org/en/14/core/metadata.html#sqlalchemy.schema.MetaData"> метаданных </a> базы данных в SQLAlchemy известны как таблица <a href="https://docs.sqlalchemy.org/en/14/core/metadata.html#sqlalchemy.schema.Table"> *Table* </a> и столбец <a href="https://docs.sqlalchemy.org/en/14/core/metadata.html#sqlalchemy.schema.Column"> *Column* </a>. Покажем, как эти объекты используются в интерфейсах *Core* и *ORM*.

<p style="font-size:150%">Создание метаданных таблиц</p>

Когда мы работаем с реляционной базой данных, одной из базовых структур для составления запросов является таблица. В SQLAlchemy «таблица» представлена объектом Python с аналогичным названием <a href="https://docs.sqlalchemy.org/en/14/core/metadata.html#sqlalchemy.schema.Table">*Table*</a>.

Чтобы начать использовать язык выражений SQLAlchemy, нам нужно создать объекты *Table*, представляющие все таблицы базы данных, с которыми мы собираемся работать. Каждая таблица может быть объявлена, то есть мы явно прописываем в исходном коде, как выглядит или **отображается** таблица, что означает, что мы генерируем объект на основе того, что уже присутствует в конкретной базе данных. Эти два подхода также можно комбинировать разными способами.

Независимо от того, будем ли мы объявлять или отображать таблицы БД, мы начинаем с коллекции, известной как объект метаданных. Этот объект, по сути, представляет собой фасад словаря Python, в котором хранится ряд объектов *Table*, привязанных к их строковым именам. Построение объекта Метаданных выглядит так:

In [12]:
from sqlalchemy import MetaData
metadata_obj = MetaData()

Наличие одного объекта *MetaData* для всего приложения является наиболее распространенным случаем. Такой объект представляется переменной уровня модуля в одном месте в приложении, как «модель» или «схема базы данных». В общем случае, в приложении может быть более одного объекта метаданных, но как правило, все объекты таблиц, связанных друг с другом, принадлежат одному объекту метаданных.

Когда у нас есть объект *MetaData*, мы можем объявить объекты Table. Рассмотрим учебную базу данных, в которой таблица пользователей будет представлять пользователей веб-сайта, а таблица адресов — список адресов электронной почты пользователей. Обычно мы присваиваем каждому объекту *Table* переменную, с помощью которой мы будем ссылаться на таблицу в коде приложения.

In [13]:
from sqlalchemy import Table, Column, Integer, String
user_table = Table(
    "user_account",
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('name', String(30)),
    Column('fullname', String)
)

Заметим, что приведенная выше конструкция таблицы похожа на оператор SQL CREATE TABLE: начинется с имени таблицы, затем перечисляется каждый столбец, указывается имя и тип данных. Объекты, которые мы используем выше:

- Таблица — представляет собой таблицу базы данных и присваивает себя коллекции метаданных.
- Столбец — представляет столбец в таблице базы данных и присваивает себя объекту таблицы. Столбец обычно включает строку имени и тип. Доступ к коллекции столбцов с точки зрения родительской таблицы обычно осуществляется через ассоциативный массив, расположенный в <a href="https://docs.sqlalchemy.org/en/14/core/metadata.html#sqlalchemy.schema.Table.c">Table.c</a>:

In [14]:
user_table.c.name

Column('name', String(length=30), table=<user_account>)

In [15]:
user_table.c.keys()

['id', 'name', 'fullname']

<a href="https://docs.sqlalchemy.org/en/14/core/type_basics.html#sqlalchemy.types.Integer">Integer</a>, <a href="https://docs.sqlalchemy.org/en/14/core/type_basics.html#sqlalchemy.types.String">String</a> — эти классы представляют типы данных SQL и могут быть указаны для столбца с ограничением на размер или без ограничений. Выше мы указали длину «30» для столбца «имя», поэтому мы создали экземпляр String (30). Но для «id» и «fullname» мы размер не указывали.

Дополнительно: <a href="https://docs.sqlalchemy.org/en/14/core/metadata.html">Описание структуры базы данных с помощью метаданных</a>.

В следующем разделе мы проиллюстрируем одну из основных функций объекта таблицы, которая заключается в выполнении команд DDL. Но сначала объявим вторую таблицу.

<p style="font-size:150%">Объявление простых ограничений</p>

Первый столбец в приведенной выше таблице *user_table* включает параметр `Column.primary_key`, который указывает на то, что этот столбец должен быть частью первичного ключа таблицы. Сам первичный ключ представлен конструкцией *PrimaryKeyConstraint*, которую мы можем видеть в атрибуте `Table.primary_key` объекта Table:

In [16]:
user_table.primary_key

PrimaryKeyConstraint(Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False))

Другое ограничение, которое чаще всего объявляется явно — это объект *ForeignKeyConstraint*, соответствующий ограничению внешнего ключа базы данных. Когда мы объявляем таблицы, которые связаны друг с другом, SQLAlchemy использует наличие этих объявлений ограничений внешнего ключа не только для того, чтобы они генерировались в операторах CREATE для базы данных, но и для помощи в построении выражений SQL.

Ограничение *ForeignKeyConstraint*, которое включает только один столбец в целевой таблице, обычно объявляется с использованием сокращенной записи на уровне столбца через объект *ForeignKey*. Ниже мы объявляем вторую таблицу — таблицу адресов, которая будет иметь ограничение внешнего ключа, ссылающееся на пользовательскую таблицу:

In [17]:
from sqlalchemy import ForeignKey
address_table = Table(
    "address",
    metadata_obj,
    Column('id', Integer, primary_key=True),
    Column('user_id', ForeignKey('user_account.id'), nullable=False),
    Column('email_address', String, nullable=False)
)

В приведенной выше таблице также представлен третий тип ограничения, которым в SQL является ограничение «NOT NULL», указанное с помощью параметра `Column.nullable`.

<Table align="left">
    <tr><td bgcolor=#adb5bd></td>
        <td style="text-align:left; font-size:10pt">
Совет.

При использовании объекта ForeignKey в определении столбца мы можем опустить тип данных - он автоматически выводится из связанного столбца; в приведенном выше примере - это тип данных Integer столбца user_account.id.
        </td></tr>
</Table>

Завершим выполнение команд DDL для таблиц пользователей и адресов.

<p style="font-size:150%">Отправка команд DDL в базу данных</p>

Мы построили иерархию объектов для представления двух таблиц базы данных, начиная с корневого объекта *MetaData*, затем два объекта *Table*, каждый из которых содержит набор объектов *Column* и *Constraint*. Эта объектная структура будет в центре большинства последующих операций, которые мы будем выполнять как с *Core*, так и с *ORM*.

Первая полезная вещь, которую мы можем сделать с этой структурой — это отправить операторы DDL CREATE TABLE в нашу базу данных SQLite. У нас уже есть все инструменты, необходимые для этого, а именно: вызов метода `MetaData.create_all()` и отправка ему движка *Engine*, который ссылается на целевую базу данных:

In [18]:
metadata_obj.create_all(engine)

2022-04-21 10:32:55,364 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:32:55,364 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2022-04-21 10:32:55,364 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 10:32:55,372 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2022-04-21 10:32:55,373 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 10:32:55,374 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-04-21 10:32:55,375 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 10:32:55,376 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-04-21 10:32:55,376 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 10:32:55,377 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-04-21 10:32:55,378 INFO sqlalchemy.engine.Engine [no key 0.00052s] ()
2022-04-21 10:32:55,379 INFO sqlalchemy.engine.Engine 
C

Процесс создания DDL по умолчанию включает некоторые специфичные для SQLite операторы PRAGMA, которые проверяют существование каждой таблицы перед отправкой CREATE. Полная последовательность шагов включена в пару BEGIN/COMMIT для поддержки транзакционного DDL (SQLite действительно поддерживает транзакционный DDL, однако драйвер базы данных sqlite3 исторически запускает DDL в режиме «автоматической фиксации»).

Процесс создания также заботится об отправке операторов CREATE в правильном порядке: ограничение FOREIGN KEY зависит от существующей пользовательской таблицы, поэтому таблица адресов создается второй. В более сложных сценариях зависимостей ограничения FOREIGN KEY также могут применяться к таблицам постфактум с использованием ALTER.

Объект *MetaData* также имеет метод `MetaData.drop_all()`, который выдает операторы DROP в обратном по отношению к CREATE порядке для удаления элементов схемы.

<p style="font-size:150%">Определение метаданных таблицы с помощью ORM</p>

В этом разделе, посвященном только ORM, будет представлен пример объявления той же структуры базы данных, что и в предыдущем разделе, с использованием парадигмы конфигурации, более ориентированной на ORM. При использовании ORM процесс объявления метаданных таблицы обычно сочетается с процессом объявления сопоставленных классов. Сопоставленный класс — это класс Python, который мы хотели бы создать, который затем будет иметь атрибуты, которые будут связаны со столбцами в таблице базы данных. Хотя существует несколько вариантов того, как это достигается, наиболее распространенный стиль известен как декларативный и позволяет нам одновременно объявлять наши определяемые пользователем классы и метаданные таблицы.

<p style="font-size:150%">Установка реестра</p>

При создании ORM коллекция метаданных содержится в объекте, называемом <a href="https://docs.sqlalchemy.org/en/14/orm/mapping_api.html#sqlalchemy.orm.registry">реестром</a>. Мы создаём реестр вот так:

In [19]:
from sqlalchemy.orm import registry
mapper_registry = registry()

После создания реестр сразу включает в себя объект метаданных, который будет содержать в себе коллекцию таблиц.

In [20]:
mapper_registry.metadata

MetaData()

Вместо того, чтобы объявлять объекты *Table* напрямую, мы теперь будем объявлять их косвенно через директивы, применяемые к нашим отображаемым классам. В наиболее распространенном подходе каждый отображаемый класс происходит от общего базового класса, известного как **декларативная база**. Получим новую декларативную базу из реестра с помощью метода `register.generate_base()`:

```
Base = mapper_registry.generate_base()
```

Совет<br>
Шаги по созданию реестра и класса декларативной базы могут быть объединены с помощью известрной функции *declarative_base()*.

In [21]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()

<p style="font-size:150%">Объявление сопоставленных классов</p>

Приведенный выше базовый объект — это класс Python, который будет служить базовым для классов ORM, которые мы объявляем. Теперь мы можем определить классы ORM для таблиц пользователей и адресов в терминах новых классов *User* и *Address*:

In [22]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey
from sqlalchemy.orm import relationship
class User(Base):
    __tablename__ = 'user_account'
    id = Column(Integer, primary_key=True)
    name = Column(String(30))
    fullname = Column(String)
    addresses = relationship("Address", back_populates="user")
    def __repr__(self):
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
    __tablename__ = 'address'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('user_account.id'))
    user = relationship("User", back_populates="addresses")
    def __repr__(self):
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

Вышеупомянутые два класса теперь являются нашими сопоставленными классами и доступны для использования в операциях запросов, которые будут описаны позже. Но они также включают в себя объекты Table, которые были сгенерированы как часть процесса декларативного сопоставления и эквивалентны тем, которые мы объявили непосредственно в предыдущем разделе Core. Мы можем увидеть эти объекты Table из декларативного сопоставленного класса, используя атрибут `.__table__`:

In [23]:
User.__table__

Table('user_account', MetaData(), Column('id', Integer(), table=<user_account>, primary_key=True, nullable=False), Column('name', String(length=30), table=<user_account>), Column('fullname', String(), table=<user_account>), schema=None)

Объект *Table* был сгенерирован в результате декларативного процесса на основе атрибута `.__tablename__`, определенного для каждого из наших классов, а также посредством использования объектов *Column*, назначенных атрибутам внутри классов. Эти объекты *Column* обычно объявляются без явного поля «имя» внутри конструктора, так как декларативный процесс присвоит имя автоматически на основе имени используемого атрибута.

<p style="font-size:120%">Другие сведения о сопоставленном классе</p>

Обратим внимание на следующие атрибуты:

Классы имеют автоматически сгенерированный метод `__init__()`, которые позволяют осуществлять параметризованное построение объектов. Мы можем также создать собственный метод `__init__()`.  `__init__()` позволяет нам создавать объекты и адреса, передавая имена атрибутов, большинство из которых связаны непосредственно с объектами *Column*, как имена параметров:

In [24]:
sandy = User(name="sandy", fullname="Sandy Cheeks")

метод `__repr__()` необязателен и предназначен лишь для того, чтобы наши пользовательские классы имели описательное строковое представление:

In [25]:
sandy

User(id=None, name='sandy', fullname='Sandy Cheeks')

Отметим, что атрибут *id* автоматически возвращает *None*, а не вызывает *AttributeError*, как этого можно было бы ожидать в Python.

Мы также включили двунаправленное отношение — это еще одна полностью необязательная конструкция,  называемая *relationship()* для обоих классов, которая указывает ORM, что классы *User* и *Address* ссылаются друг на друга в соотношении один ко многим.

<p style="font-size:150%">Отправка DDL в базу данных</p>

Выполнение команды DDL с сопоставленными классами ORM ничем не отличается от аналогичного действия с помощью интерейса Core. Если бы мы хотели выполнить команды DDL для объектов *Table*, которые мы создали как часть наших декларативно сопоставленных классов, мы по-прежнему могли бы использовать `MetaData.create_all()`.

В нашем случае мы уже создали в базе данных таблицы пользователей и адресов. Если бы мы этого еще не сделали, мы могли бы свободно использовать метаданные, связанные с нашим реестром, и декларативный базовый класс ORM, чтобы сделать это, используя `MetaData.create_all()`:

```
# выполнение команды CREATE statements с конкретным реестром ORM
mapper_registry.metadata.create_all(engine)
Base.metadata.create_all(engine)
```

<p style="font-size:150%">Совместное использование интерфейсов Core и ORM Declarative</p>

В качестве альтернативного подхода к процессу сопоставления, показанному ранее в разделе «Объявление сопоставленных классов», мы также можем использовать объекты таблиц, которые мы создали непосредственно в разделе «Создание метаданных таблиц», в сочетании с декларативными сопоставленными классами из сгенерированного `declarative_base()` базового класса.

Такой подход называют гибридным, и он состоит в прямом назначении атрибута `.__table__`, а не в декларативном процессе:

```
mapper_registry = registry()
Base = mapper_registry.generate_base()

class User(Base):
    __table__ = user_table

    addresses = relationship("Address", back_populates="user")

    def __repr__(self):
        return f"User({self.name!r}, {self.fullname!r})"

class Address(Base):
    __table__ = address_table

    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return f"Address({self.email_address!r})"
```

Эти два класса эквивалентны классам, созданным в прдыдущем примере раздела "Объявление сопоставленных классов".

<p style="font-size:150%">Отображение таблиц</p>

Отображение таблицы относится к процессу чтения текущего состояния базы данных. В предыдущих разделах мы объявляли табличные объекты в Python, а затем отправляли команды DDL в базу данных. Процесс отображения делает это в обратном порядке.

В качестве примера отражения создадим новый объект таблицы, отображающий таблицу *some_table*, которую мы создали вручную в более ранних разделах этого документа. Используем конструктор таблицы, укажем имя таблицы и объект метаданных, к которым она будет принадлежать. Затем, вместо того, чтобы создавать столбцы и ограничения, передадим в конструктор целевой движок с помощью параметра `Table.autoload_with`:

In [26]:
some_table = Table("some_table", metadata_obj, autoload_with=engine)

2022-04-21 10:41:17,190 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:41:17,191 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2022-04-21 10:41:17,191 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 10:41:17,193 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-04-21 10:41:17,193 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2022-04-21 10:41:17,195 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2022-04-21 10:41:17,195 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 10:41:17,196 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2022-04-21 10:41:17,197 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-21 10:41:17,198 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
20

По завершении этого процесса объект *some_table* содержит информацию о столбцах этой таблицы. Теперь этот объект можно использовать также как и другие таблицы, объявленные явно:

In [27]:
some_table

Table('some_table', MetaData(), Column('x', INTEGER(), table=<some_table>), Column('y', INTEGER(), table=<some_table>), schema=None)

Более подробно об отображении таблиц в <a href="https://docs.sqlalchemy.org/en/14/core/reflection.html">документации</a>.

[К оглавлению](#Home)

<a id=4></a>

# 4. Работа с данными: INSERT, UPDATE, DELETE, SELECT

<p style="font-size:150%; font-weight:bold">4.1. Вставка с помощью Core</p>

Для вставки строк с помощью интерфейса Core используем функцию *insert()*.

In [28]:
from sqlalchemy import insert
stmt = insert(user_table).values(name='vivaldi', fullname='Antonio Vivaldi')

Это SQL-выражение можно представить в текстовом виде:

In [29]:
print(stmt)

INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)


Метод <a href="https://docs.sqlalchemy.org/en/14/core/foundation.html#sqlalchemy.sql.expression.ClauseElement.compile">ClauseElement.compile()</a> возвращает компилированный объект. Если применить его к объекту SQL-выражения, получим

In [30]:
stmt.compile()

Функция *str()* возвращает текстовое представление объекта.

In [31]:
str(stmt.compile())

'INSERT INTO user_account (name, fullname) VALUES (:name, :fullname)'

Свойство *params* показывает значение параметров SQL-выражения.

In [32]:
stmt.compile().params

{'name': 'vivaldi', 'fullname': 'Antonio Vivaldi'}

Выполним SQL-выражение.

In [33]:
with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

2022-04-21 10:42:07,156 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:42:07,157 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-04-21 10:42:07,158 INFO sqlalchemy.engine.Engine [generated in 0.00157s] ('vivaldi', 'Antonio Vivaldi')
2022-04-21 10:42:07,159 INFO sqlalchemy.engine.Engine COMMIT


В приведенной выше простой форме оператор INSERT возвращает целочисленное значение первичного ключа, которое мы можем получить с помощью метода доступа CursorResult.inserted_primary_key:

In [34]:
result.inserted_primary_key

(1,)

Полученное значение является кортежем, так как в общем случае первичный ключ может состоять из более чем одного атрибута. Начиная с версии 1.4.8 возвращаемый с помощью *inserted_primary_key* кортеж является именованным, представляет собой объект типа строки *Row*:

In [35]:
row = result.inserted_primary_key
print(row._mapping)

{'id': 1}


В приведенном выше примере использовался метод `Insert.values()` для явного создания предложения VALUES инструкции SQL INSERT. На самом деле у этого метода есть несколько вариантов, которые допускают специальные формы, такие как несколько строк в одном выражении и вставка выражений SQL. Однако обычный способ использования Insert заключается в том, что предложение VALUES генерируется автоматически из параметров, переданных методу Connection.execute(); ниже мы вставляем еще две строки, чтобы проиллюстрировать это:

In [36]:
with engine.connect() as conn:
    result = conn.execute(
        insert(user_table),
        [
            {"name": "colonel", "fullname": "Harland David Sanders"},
            {"name": "bond", "fullname": "James Bond"}
        ]
    )
    conn.commit()

2022-04-21 10:42:21,077 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:42:21,078 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?)
2022-04-21 10:42:21,078 INFO sqlalchemy.engine.Engine [generated in 0.00150s] (('colonel', 'Harland David Sanders'), ('bond', 'James Bond'))
2022-04-21 10:42:21,079 INFO sqlalchemy.engine.Engine COMMIT


Этот код имеет форму «executemany», ранее использованную нами в подразделе «Отправка нескольких параметров» с помощью конструкции *text()*, однако, теперь нам не потребовалось создавать команду SQL в текстовом виде. Передавая словарь или список словарей в метод `Connection.execute()` в сочетании с конструкцией *Insert*, *Connection* гарантирует, что передаваемые имена столбцов будут автоматически выражены в предложении *VALUES* команды *Insert*.

<p style="font-size:150%">INSERT…FROM SELECT</p>

Конструкция *Insert* может составлять SQL-выражение, в котором команда INSERT получает строки непосредственно из SELECT с помощью метода `Insert.from_select()`:

In [37]:
from sqlalchemy import select

select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com") # Получаем строки таблицы user_account

# Вставляем в таблицу address
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt)

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account


In [38]:
with engine.connect() as conn:
    result = conn.execute(insert_stmt)
    conn.commit()

2022-04-21 10:42:27,077 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:42:27,078 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || ? AS anon_1 
FROM user_account
2022-04-21 10:42:27,079 INFO sqlalchemy.engine.Engine [generated in 0.00157s] ('@aol.com',)
2022-04-21 10:42:27,080 INFO sqlalchemy.engine.Engine COMMIT


<p style="font-size:150%">INSERT…RETURNING</p>

Предложение RETURNING для поддерживаемых серверов БД используется автоматически для извлечения последнего вставленного значения первичного ключа, а также значений по умолчанию для сервера. Однако предложение RETURNING также может быть указано явно с помощью метода `Insert.returning()`, в этом случае объект Result содержит возвращаемые значения:

In [39]:
insert_stmt = insert(address_table).returning(address_table.c.id, address_table.c.email_address)
print(insert_stmt)

INSERT INTO address (id, user_id, email_address) VALUES (:id, :user_id, :email_address) RETURNING address.id, address.email_address


Предложение RETURNING можно комбинировать с FROM SELECT

In [40]:
select_stmt = select(user_table.c.id, user_table.c.name + "@aol.com")
insert_stmt = insert(address_table).from_select(
    ["user_id", "email_address"], select_stmt
)
print(insert_stmt.returning(address_table.c.id, address_table.c.email_address))

INSERT INTO address (user_id, email_address) SELECT user_account.id, user_account.name || :name_1 AS anon_1 
FROM user_account RETURNING address.id, address.email_address


<p style="font-size:150%; font-weight:bold">4.2. Селекция строк с помощью Core или ORM</p>

И для Core, и для ORM функция select() генерирует конструкцию Select, которая используется для всех запросов SELECT. Передаваемый таким методам, как Connection.execute() в Core и `Session.execute()` в ORM, оператор SELECT выполняется в текущей транзакции, а строки результатов доступны через возвращаемый объект Result.

Функция *select()* позволяет создать SQL-выражение аналогично тому, как это было рассмотрено выше для функции *insert()*.

In [41]:
from sqlalchemy import select
stmt = select(user_table).where(user_table.c.name == 'colonel')
print(stmt)

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


Для выполнения выполнения SQL-выражения мы передаём его в исполняемый метод. Выражение с использованием команды SELECT возвращает строки, по которым можно совершить итерацию.

In [42]:
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row)

2022-04-21 10:42:43,204 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:42:43,205 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-04-21 10:42:43,206 INFO sqlalchemy.engine.Engine [generated in 0.00144s] ('colonel',)
(2, 'colonel', 'Harland David Sanders')
2022-04-21 10:42:43,207 INFO sqlalchemy.engine.Engine ROLLBACK


При использовании ORM мы выполняем команду *select()* ее с помощью метода `Session.execute()` в сеансе. Используя этот подход, мы продолжаем получать объекты Row из результата, однако эти строки теперь могут включать экземпляры класса User как отдельные элементы в каждой строке:

In [43]:
stmt = select(User).where(User.name == 'vivaldi')
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)

2022-04-21 10:42:46,028 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:42:46,030 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2022-04-21 10:42:46,032 INFO sqlalchemy.engine.Engine [generated in 0.00110s] ('vivaldi',)
(User(id=1, name='vivaldi', fullname='Antonio Vivaldi'),)
2022-04-21 10:42:46,033 INFO sqlalchemy.engine.Engine ROLLBACK


<p style="font-size:150%">Определение предложений COLUMNS и FROM</p>

Функция *select()* принимает позиционные элементы, представляющие любое количество выражений *Column* и/или *Table*, а также широкий спектр совместимых объектов, которые преобразуются в список SQL-выражений. Эти элементы также служат  для создания предложения FROM, которое неявно выводится из переданных столбцов и табличных выражений.

In [44]:
print(select(user_table))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


Чтобы выбрать данные из отдельных столбцов с использованием подхода *Core*, доступ к объектам столбцов осуществляется с помощью `Table.c`. Предложение FROM будет сформировано как набор всех объектов *Table*.

In [45]:
print(select(user_table.c.name, user_table.c.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


<p style="font-size:150%">Выбор сущностей и столбцов с помощью ORM</p>

Выберем атрибуты класса *User*.

In [46]:
print(select(User))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account


Далее, если выполнить это выражение с помощью метода `session.execute()`, мы получим объекты строк как 
экземпляры класса *User*.

In [47]:
row = session.execute(select(User)).first()
row

2022-04-21 10:42:57,956 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:42:57,957 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-04-21 10:42:57,958 INFO sqlalchemy.engine.Engine [generated in 0.00062s] ()


(User(id=1, name='vivaldi', fullname='Antonio Vivaldi'),)

Эта строка содержит всего один элемент, представляющий сущность *User*. 

In [48]:
row[0]

User(id=1, name='vivaldi', fullname='Antonio Vivaldi')

Вместо выбора всех столбцов, мы можем указать отдельные атрибуты класса.

In [49]:
print(select(User.name, User.fullname))

SELECT user_account.name, user_account.fullname 
FROM user_account


Выполним выражение.

In [50]:
row = session.execute(select(User.name, User.fullname)).first()
row

2022-04-21 10:43:05,188 INFO sqlalchemy.engine.Engine SELECT user_account.name, user_account.fullname 
FROM user_account
2022-04-21 10:43:05,189 INFO sqlalchemy.engine.Engine [generated in 0.00090s] ()


('vivaldi', 'Antonio Vivaldi')

Покажем применение этой техники для выбора атрибутов из двух таблиц - пользователей и адресов.

In [51]:
session.execute(
    select(User.name, Address).
    where(User.id==Address.user_id).
    order_by(Address.id)
).all()

2022-04-21 10:43:07,877 INFO sqlalchemy.engine.Engine SELECT user_account.name, address.id, address.email_address, address.user_id 
FROM user_account, address 
WHERE user_account.id = address.user_id ORDER BY address.id
2022-04-21 10:43:07,878 INFO sqlalchemy.engine.Engine [generated in 0.00092s] ()


[('vivaldi', Address(id=1, email_address='vivaldi@aol.com')),
 ('colonel', Address(id=2, email_address='colonel@aol.com')),
 ('bond', Address(id=3, email_address='bond@aol.com'))]

In [52]:
users = session.execute(select(User)).all()
users

2022-04-21 10:43:10,716 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account
2022-04-21 10:43:10,717 INFO sqlalchemy.engine.Engine [cached since 12.76s ago] ()


[(User(id=1, name='vivaldi', fullname='Antonio Vivaldi'),),
 (User(id=2, name='colonel', fullname='Harland David Sanders'),),
 (User(id=3, name='bond', fullname='James Bond'),)]

<p style="font-size:150%">Селекция с использованием метки</p>

Метод `ColumnElement.label()`, а также одноименный метод, доступный для атрибутов ORM, создаёт метку столбца или SQL-выражения, позволяя ему иметь конкретное имя в результирующем наборе.

In [53]:
from sqlalchemy import func, cast
stmt = (
    select(
        ("Username: " + user_table.c.name).label("username"),
    ).order_by(user_table.c.name)
)

with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(row.username)

2022-04-21 10:43:19,502 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:43:19,503 INFO sqlalchemy.engine.Engine SELECT ? || user_account.name AS username 
FROM user_account ORDER BY user_account.name
2022-04-21 10:43:19,503 INFO sqlalchemy.engine.Engine [generated in 0.00159s] ('Username: ',)
Username: bond
Username: colonel
Username: vivaldi
2022-04-21 10:43:19,504 INFO sqlalchemy.engine.Engine ROLLBACK


См. также: <a href="https://docs.sqlalchemy.org/en/14/tutorial/data_select.html#tutorial-order-by-label">Сортировка и группировка с помощью меток</a>

<p style="font-size:150%">Селекция с использованием текстовых выражений</p>

Когда мы создаём объект селекции *Select* с помощью функции *select()*, мы обычно передаём ей последовательность объектов таблиц и столбцов, определенных в метаданных. В ходе селекции с помощью объектно-ориентированного подхода ORM мы передаём сопоставленные атрибуты, преставляющие столбцы. Однако, некоторые части SQL-выражения, такие как строковые константы, удобнее формировать в текстовом виде. Для этого можно использовать функцию *text()*, введенную ранее. Следующий пример иллюстрирует использование текстовой константы в запросе.

In [54]:
from sqlalchemy import text
stmt = (
    select(
        text("'username'"), user_table.c.name
    ).order_by(user_table.c.name)
)
with engine.connect() as conn:
    print(conn.execute(stmt).all())

2022-04-21 10:43:44,221 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:43:44,222 INFO sqlalchemy.engine.Engine SELECT 'username', user_account.name 
FROM user_account ORDER BY user_account.name
2022-04-21 10:43:44,223 INFO sqlalchemy.engine.Engine [generated in 0.00143s] ()
[('username', 'bond'), ('username', 'colonel'), ('username', 'vivaldi')]
2022-04-21 10:43:44,224 INFO sqlalchemy.engine.Engine ROLLBACK


Хотя конструкция *text()* может использоваться в большинстве случаев для буквального ввода команд SQL, чаще всего мы фактически имеем дело с текстовыми единицами, каждая из которых представляет отдельное выражение столбца. В этом распространенном случае мы можем получить больше функциональности из нашего текстового фрагмента, используя вместо этого конструкцию *literal_column()*. Этот объект похож на *text()*, за исключением того, что вместо произвольного SQL-запроса в любой форме он явно представляет один «столбец», который затем может быть помечен и на него можно ссылаться в подзапросах и других выражениях:

In [55]:
from sqlalchemy import literal_column
stmt = (
    select(
        literal_column("'username'").label("p"), user_table.c.name.label("q")
    ).order_by(user_table.c.name)
)
with engine.connect() as conn:
    for row in conn.execute(stmt):
        print(f"{row.p}, {row.q}")

2022-04-21 10:43:49,445 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-21 10:43:49,446 INFO sqlalchemy.engine.Engine SELECT 'username' AS p, user_account.name AS q 
FROM user_account ORDER BY user_account.name
2022-04-21 10:43:49,447 INFO sqlalchemy.engine.Engine [generated in 0.00166s] ()
username, bond
username, colonel
username, vivaldi
2022-04-21 10:43:49,448 INFO sqlalchemy.engine.Engine ROLLBACK


<p style="font-size:150%">Предложение WHERE</p>

Предложение WHERE позволяет устанавливать условие (предикат) на отбор строк отношения.

In [56]:
clause_1 = (user_table.c.name == 'vivaldi')
clause_2 = (address_table.c.user_id > 10)

print(select(user_table).where(clause_1))

SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = :name_1


Для добавления условий, выполняющихся одновременно, метод *where()* можно повторять неограниченное число раз.

In [57]:
print(
    select(address_table.c.email_address).
    where(user_table.c.name == 'vivaldi').
    where(address_table.c.user_id == user_table.c.id)
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


Также, один метод *where()* может содержать множество условий.

In [58]:
print(
    select(address_table.c.email_address).
    where(
         user_table.c.name == 'vivaldi',
         address_table.c.user_id == user_table.c.id
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE user_account.name = :name_1 AND address.user_id = user_account.id


Логическое сложение OR и умножение AND можно применить, если использовать <a href="https://docs.sqlalchemy.org/en/14/core/sqlelement.html#sqlalchemy.sql.expression"> функции </a> *or_()* и *and_()*.

In [59]:
# Получить почтовые адреса пользователей с именами vivaldin и bond.
from sqlalchemy import and_, or_
print(
    select(Address.email_address).
    where(
        and_(
            or_(User.name == 'vivaldi', User.name == 'bond'),
            Address.user_id == User.id
        )
    )
)

SELECT address.email_address 
FROM address, user_account 
WHERE (user_account.name = :name_1 OR user_account.name = :name_2) AND address.user_id = user_account.id


Для простых сравнений «на равенство» с одним объектом также есть популярный метод, известный как Select.filter_by(), который принимает аргументы ключевого слова, соответствующие ключам столбцов или именам атрибутов ORM.

In [ ]:
print(
    select(User).filter_by(name='vivaldi', fullname='Antonio Vivaldi')
)

См. также: <a href="https://docs.sqlalchemy.org/en/14/core/operators.html">использовнаие операторов в SQLAlchemy</a>

<p style="font-size:150%">Предложение FROM в явном виде и соединение JOIN</p>

Как мы видели ранее, предложение FROM обычно выводится на основе выражений, которые мы устанавливаем в предложении столбцов, а также других элементов Select. То есть, если мы устанавливаем один столбец из определенной таблицы в предложении COLUMNS, то эта таблица помещается в предложение FROM:

In [60]:
print(select(user_table.c.name))

SELECT user_account.name 
FROM user_account


Так же происходит, когда мы отбираем данные из столбцов, принадлежащих разным таблицам.

In [61]:
print(select(user_table.c.name, address_table.c.email_address))

SELECT user_account.name, address.email_address 
FROM user_account, address


Чтобы СОЕДИНИТЬ эти две таблицы вместе, мы обычно используем один из двух методов в Select. Первый — это метод `Select.join_from()`, который позволяет нам явно указать левую и правую стороны JOIN:

In [62]:
print(
    select(user_table.c.name, address_table.c.email_address).
    join_from(user_table, address_table)
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


Другим является метод `Select.join()`, который указывает только правую часть JOIN, левая сторона выводится неявно:

In [63]:
print(
    select(user_table.c.name, address_table.c.email_address).
    join(address_table)
)

SELECT user_account.name, address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


В этих примерах условие соединения ON сформировано неявно. Однако, его можно указывать в явном виде.

У нас также есть возможность явно добавлять элементы в предложение FROM. Для этого в следующем примере мы используем метод `Select.select_from()` чтобы указать первую таблицу соединения *user_account* и далее `Select.join()`.

In [64]:
print(
    select(address_table.c.email_address).
    select_from(user_table).join(address_table)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


Другой пример, когда может понадобиться использование метода *select_from()* - недостаток информации об используемых таблицах.

In [65]:
from sqlalchemy import func
print (
    select(func.count('*')).select_from(user_table)
)

SELECT count(:count_2) AS count_1 
FROM user_account


Больше примеров в <a href="https://docs.sqlalchemy.org/en/14/orm/queryguide.html">ORM Querying Guide</a>

<p style="font-size:150%">Формирование условия соединения</p>

Предыдущие примеры использования соединения JOIN показали, что конструкция Select может соединять две таблицы и автоматически создавать предложение ON. Так происходило, потому что таблица *address* содержит ограничение ForeignKeyConstraint, которое обеспечивает ссылку на таблицу *user_account*.

Если левая и правая стороны соединения не имеют такого ограничения или есть несколько ограничений, нам нужно указать предложение ON напрямую. В этом случае `Select.join()`, и `Select.join_from()` принимают дополнительный аргумент для предложения ON, который задается с использованием той же механики SQL-выражения, что мы видели в предложении WHERE:

In [66]:
print(
    select(address_table.c.email_address).
    select_from(user_table).
    join(address_table, user_table.c.id == address_table.c.user_id)
)

SELECT address.email_address 
FROM user_account JOIN address ON user_account.id = address.user_id


<p style="font-size:150%">Внешнее и полное соединения</p>

Оба метода `Select.join()` и `Select.join_from()` принимают аргументы ключевых слов `Select.join.isouter` и `Select.join.full`, которые позволяют создавать предложения левого внешнего LEFT OUTER JOIN и полного FULL OUTER JOIN соединений:

In [ ]:
print(
    select(user_table).join(address_table, isouter=True)
)


print(
    select(user_table).join(address_table, full=True)
)

Существует также метод `Select.outerjoin()`, эквивалентный использованию `.join(..., isouter=True)`.

<Table align="left">
    <tr><td bgcolor=#adb5bd></td>
        <td style="text-align:left; font-size:10pt">
Примечание.

SQLAlchemy не поддерживает команду RIGHT OUTER JOIN. Вместо этого поменяйте местами таблицы и используйте LEFT OUTER JOIN.
        </td></tr>
</Table>